<a href="https://colab.research.google.com/github/Pmilivojevic/Colorization/blob/master/UNet_ResNet_3_to_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Activation, Dropout, Add, Concatenate, Input, UpSampling2D, Flatten, BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
image_size = 512
batch_size = 1

In [ ]:
data_path= '/content/drive/My Drive/Colab Notebooks/train_36'

train= []
train_labels= []

for i in range(len(os.listdir(data_path))):
  if i >= 0 and i <200:
    img= cv2.imread(os.path.join(data_path, os.listdir(data_path)[i]))
    train_labels.append(cv2.cvtColor(cv2.resize(img, (512, 512), cv2.INTER_AREA), cv2.COLOR_BGR2RGB))

In [ ]:
i= 0
for img in train_labels:
  gray= cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  train.append(cv2.merge((gray, gray, gray)))

train= np.array(train).reshape(-1, 512, 512, 3)
train= train/255.0
train_labels= np.array(train_labels).reshape(-1, 512, 512, 3)
train_labels= train_labels/255.0

In [ ]:
test_data_path= '/content/drive/My Drive/Colab Notebooks/test'

test= []
test_labels= []

for i in range(len(os.listdir(test_data_path))):
  img= cv2.imread(os.path.join(test_data_path, os.listdir(test_data_path)[i]))
  test_labels.append(cv2.cvtColor(cv2.resize(img, (512, 512), cv2.INTER_AREA), cv2.COLOR_BGR2RGB))

In [ ]:
i= 0
for img in test_labels:
  gray= cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  test.append(cv2.merge((gray, gray, gray)))

test= np.array(test).reshape(-1, 512, 512, 3)
test= test/255.0
test_labels= np.array(test_labels).reshape(-1, 512, 512, 3)
test_labels= test_labels/255.0

In [ ]:
'''def bn_act(x, act=True):
    x = BatchNormalization()(x)
    if act == True:
        x = Activation("relu")(x)
    return x

def conv_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = bn_act(x)
    conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
    return conv

def stem(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    conv = conv_block(conv, filters, kernel_size=kernel_size, padding=padding, strides=strides)

    shortcut = Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)

    output = Add()([conv, shortcut])
    return output

def residual_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    res = conv_block(x, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    res = conv_block(res, filters, kernel_size=kernel_size, padding=padding, strides=1)

    shortcut = Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)

    output = Add()([shortcut, res])
    return output

def upsample_concat_block(x, xskip):
    u = UpSampling2D((2, 2))(x)
    c = Concatenate()([u, xskip])
    return c'''

In [ ]:
'''def ResUNet():
    f = [16, 32, 64, 128, 256]
    inputs = Input((image_size, image_size, 3))

    ## Encoder
    e0 = inputs
    e1 = stem(e0, f[0])
    e2 = residual_block(e1, f[1], strides=2)
    e3 = residual_block(e2, f[2], strides=2)
    e4 = residual_block(e3, f[3], strides=2)
    e5 = residual_block(e4, f[4], strides=2)

    ## Bridge
    b0 = conv_block(e5, f[4], strides=1)
    b1 = conv_block(b0, f[4], strides=1)

    ## Decoder
    u1 = upsample_concat_block(b1, e4)
    d1 = residual_block(u1, f[4])

    u2 = upsample_concat_block(d1, e3)
    d2 = residual_block(u2, f[3])

    u3 = upsample_concat_block(d2, e2)
    d3 = residual_block(u3, f[2])

    u4 = upsample_concat_block(d3, e1)
    d4 = residual_block(u4, f[1])

    outputs = Conv2D(3, (1, 1), padding="same", activation="sigmoid")(d4)
    model = Model(inputs, outputs)
    return model'''

In [ ]:
def r2_score(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true)))
    return ( 1 - SS_res/(SS_tot + K.epsilon()))

In [ ]:
'''model = ResUNet()
adam = Adam()
model.compile(Adam(lr= .0001), loss= 'mean_squared_error', metrics=[r2_score])
model.summary()'''

In [ ]:
models_path= '/content/drive/My Drive/Colab Notebooks/models/colorization_3to3_chanels'
len(os.listdir(models_path))
#model_load_path= models_path + '/model_' + str(len(os.listdir(models_path))) + '.h5'
model_load_path= models_path + '/model_10.h5'
print(model_load_path)

In [ ]:
dependencies= {
    'r2_score': r2_score
}
model= load_model(model_load_path, custom_objects= dependencies)
#model.summary()

In [ ]:
model.optimizer= Adam(learning_rate= 0.001)

In [ ]:
model_save_path= models_path + '/model_' + str(len(os.listdir(models_path))+1) + '.h5'
model.fit(train, train_labels, batch_size= batch_size, validation_split= 0.1, epochs=50)
model.save(model_save_path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#i= np.reshape(test[5], (512, 512))
plt.imshow(test_labels[67], cmap= 'gray')

In [ ]:
predictions= model.predict(test)

In [ ]:
plt.imshow(predictions[67])

In [ ]:
plt.imshow(test[60], cmap= 'gray')

In [ ]:
test_image= cv2.imread('/content/drive/My Drive/Colab Notebooks/Sofija.bmp')
test_image= test_image[70:442, 70:442]
test_image= cv2.resize(test_image, (512, 512), cv2.INTER_AREA)
test_image.shape

In [ ]:
plt.imshow(test_image)

In [ ]:
#test_image= cv2.cvtColor(test_image, cv2.COLOR_RGB2GRAY)
test_image= np.array(test_image).reshape(-1, 512, 512, 3)
test_image= test_image/255.0

In [ ]:
prediction= model.predict(test_image)

In [ ]:
plt.imshow(prediction[0])

In [ ]:
B_model_load_path= models_path + '/model_16.h5'
B_model= load_model(B_model_load_path, custom_objects= dependencies)
B_prediction= B_model.predict(test_image)
#img= cv2.cvtColor(cv2.merge((np.uint8(test_image[0,:,:,0]*255), np.uint8(B_prediction[0,:,:,0]*255), np.uint8(B_prediction[0,:,:,1]*255))), cv2.COLOR_LAB2RGB)
plt.imshow(B_prediction[0], cmap= 'gray')